In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from os import listdir
from os.path import join

In [2]:
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

def load_documents(folder):
    files = [file for file in listdir("library") if file.endswith(".pdf")]
    docs = [PyPDFLoader(join(folder, file)).load() for file in files]
    return [item for sublist in docs for item in sublist]

In [3]:
class RAGApplication:

    def __init__(self, folder="library"):

        self.embedder = OllamaEmbeddings(model="qwen3-embedding:8b", temperature=0)
        self.llm = ChatOllama(model="llama3.1:8b", temperature=0)

        self.rag_chain = prompt | self.llm | StrOutputParser()

        self.text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=250,
            chunk_overlap=0
        )

        self.docs_list = load_documents(folder=folder)
        self.vectorstore = self.create_vectorstore()
        self.retriever = self.vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "k": 5,
            }
        )

    def create_vectorstore(self):

        return SKLearnVectorStore.from_documents(
            documents=self.text_splitter.split_documents(self.docs_list),
            embedding=self.embedder,
        )

    def run(self, question):

        documents = self.retriever.invoke(question)
        documents = [doc.page_content for doc in documents]
        documents = "\\n".join(documents)

        answer = self.rag_chain.invoke({
            "question": question,
            "documents": documents
        })

        return answer

In [4]:
rag_app = RAGApplication()

In [5]:
print(rag_app.run("Welche Anforderungen an Bachelorarbeiten gibt es?"))

Ich kann keine Antwort finden, da das Dokument keine Anforderungen an Bachelorarbeiten enthält. Es scheint jedoch, dass es um andere Themen wie Masterarbeiten und Abschlussprüfungen für Bachelorstudiengänge geht.

Wenn ich jedoch auf die ersten beiden Dokumente zurückgreifen darf, kann ich sagen, dass es folgende Anforderungen an Bachelorarbeiten gibt:

* Die Arbeit muss ohne fremde Hilfe selbständig verfasst werden.
* Nur die angegebenen Quellen und Hilfsmittel dürfen verwendet werden.
* Die Arbeit muss fristgerecht und entsprechend den Richtlinien abgegeben werden.

Es ist jedoch nicht klar, ob diese Anforderungen speziell für Bachelorarbeiten gelten oder allgemein für alle Arbeiten an der Hochschule.


In [6]:
print(rag_app.run("Wieviele Seiten darf eine Masterarbeit umfassen?"))

Eine Masterarbeit darf umfassen:

10.000 bis 15.000 Wörter (+/- 10% Abweichung erlaubt)

Im inhaltlich begründeten Einzelfall kann mit Zustimmung der Studiengangsleitung von diesem Umfang abgewichen werden.


In [7]:
print(rag_app.run("Wer darf den Titel Honorarprofessor führen?"))

Wer darf den Titel Honorarprofessor führen?

Laut dem Dokument kann der Titel Honorarprofessor von Personen geführt werden, die als Honorarprofessoren (FH) oder Honorarprofessorinnen (FH) bezeichnet werden. Dies wird in Kapitel 3 des Dokuments beschrieben und besagt, dass die Honorarprofessur (FH) zeitlich für die Dauer der aktiven Lehrtätigkeit beschränkt ist.
